#### the faces' latent space is created with pbaylies' StyleGAN encoder (building on original work from Puzer): 

pbaylies repo: https://github.com/pbaylies/stylegan-encoder
StyleGAN paper: https://arxiv.org/abs/1812.04948

#### the latent space can then be navigated with tr1pzz's fork of the original ShenYujun's InterFaceGAN

Original Repo: https://github.com/ShenYujun/InterFaceGAN 
Paper: https://arxiv.org/abs/1907.10786


https://colab.research.google.com/github/iyaja/stylegan-encoder/blob/master/generate_GoT_characters_with_StyleGAN.ipynb

# context

In [0]:
# GLOBALS


autoDownloadImages = 0 # auto run mode - get uri[uri.rfind("/")+1:] files from imgsToAutodownload
uploadRawImages = 0 # manually upload raw images
takePhotoImages = 0 # take photo for raw images
photosInDrive = 0 # get photos from drive
pictsInEncoder = 1 # assume images in folder

imgsToAutodownload = [
  "https://github.com/sifbuilder/eodoes/raw/master/packages/eodoes-eodo-eofaces/img/amgc.jpg",
  "https://github.com/sifbuilder/eodoes/raw/master/packages/eodoes-eodo-eofaces/img/amgf.jpg"
]

downloadResults = 0 # download results

emptyspace = 0 # or reset runtime
toSmptyFolders = ["stylegan-encoder", "InterFaceGAN", "output_vectors.npy"]

usegdrive = 0 # would mount gdrive
displayInVideo = 1 # would scape video output

In [0]:
# would mount gdrive

if usegdrive == 1:
  from google.colab import drive
  drive.mount('/content/gdrive')

  # https://medium.com/@prajwal.prashanth22/google-colab-drive-as-persistent-storage-for-long-training-runs-cb82bc1d5b71
  from keras.callbacks import *
  filepath = "/content/gdrive/My Drive/" + cnnid + "/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
  checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
  callbacks_list = [checkpoint]
  
  # referring to callbacks_list when calling model.fit
  # model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
  #                   epochs=epochs,
  #                   verbose=1,
  #                   validation_data=(x_test, y_test),
  #                   callbacks=callbacks_list)
  
  # to resume run model.load_weights at 47th epoch having reached max validation accuracy of 90.5% :
  # epoch = 47
  # acc = 0.905
  # model.load_weights("/content/gdrive/My Drive/" + cnnid + "/epochs:" + epoch + "-val_acc:" + acc + ".hdf5")
  
  

In [0]:
# requirements

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import humanize
import psutil
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
printm() 


# runtime type must be GPU enabled
import tensorflow as tf
tf.VERSION
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


import pickle
import PIL.Image
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


In [0]:
# file system context

# stylegan folder signature

stylegansign = 'encode_images.py'
isExist = os.path.exists(stylegansign)
if isExist:
  print("in stylegan. up to root")
  %cd ..  
  
  
# InterFaceGAN folder signature

interfacegansign = 'edit.py'
isExist = os.path.exists(interfacegansign)
if isExist:
  print("in InterFaceGAN. up to root")
  %cd ..  
  
!pwd  

#  encoder

In [0]:
# Clone pbaylies' stylegan encoder


path = 'stylegan-encoder'
isExist = os.path.exists(path)
if not isExist:
  print("stylegan-encoder does not exist. will clone")
  !git clone https://github.com/pbaylies/stylegan-encoder
else:
  print("stylegan-encoder already exists. will not clone")
  
# cd into the encoder repo folder  
import os
os.chdir("stylegan-encoder")  


if os.path.exists('aligned_images') is False:
    os.mkdir('aligned_images')
    
if os.path.exists('raw_images') is False:
    os.mkdir('raw_images')
    
if os.path.exists('generated_images') is False:
    os.mkdir('generated_images')
    
if os.path.exists('cache') is False:
    os.mkdir('cache')



In [0]:
# move to stylegan folder

if os.path.exists('stylegan-encoder') is True:
    os.chdir('stylegan-encoder')

!pwd    


In [0]:
# Download a pretrained resnet encoder: takes an image as input and estimates its latent code

path = 'finetuned_resnet.h5'
isExist = os.path.exists(path)
if not isExist:
  !gdown https://drive.google.com/uc?id=1aT59NFy9-bNyXjDuZOTMl0qX0jmZc6Zb
  !mkdir -p data
  !cp finetuned_resnet.h5 data
else:
  print("finetuned_resnet already exists. will not gdown it")  
  

In [0]:
# dnnlib

import dnnlib
import dnnlib.tflib as tflib

# from encoder.generator_model import Generator

# images

In [0]:
!mkdir -p raw_images

## upload

In [0]:
# Opt 1: get the images from the imgsToAutodownload global array set above 
if autoDownloadImages == 1:
  for uri in imgsToAutodownload:
    print("uri: %s" %uri)
    !wget $uri
    fname=uri[uri.rfind("/")+1:]
    print("fname: %s" %fname)            
    !mv $fname raw_images/


In [0]:
# Opt 2: upload images from the file system
if uploadRawImages == 1:
  print("upload raw images and move to raw_images")
  from google.colab import files
  uploaded = files.upload()
  for fname in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fname, length=len(uploaded[fname])))
    !mv $fname 'raw_images/'


In [0]:
# Opt 3: take image from the web cam
if takePhotoImages == 1:
  from base64 import b64decode
  from IPython.display import HTML, Audio
  from google.colab.output import eval_js
  from PIL import Image
  from datetime import datetime

  VIDEO_HTML = """
  <video autoplay
   width=%d height=%d style='cursor: pointer;'></video>
  <script>

  var video = document.querySelector('video')

  navigator.mediaDevices.getUserMedia({ video: true })
    .then(stream=> video.srcObject = stream)

  var data = new Promise(resolve=>{
    video.onclick = ()=>{
      var canvas = document.createElement('canvas')
      var [w,h] = [video.offsetWidth, video.offsetHeight]
      canvas.width = w
      canvas.height = h
      canvas.getContext('2d')
            .drawImage(video, 0, 0, w, h)
      video.srcObject.getVideoTracks()[0].stop()
      video.replaceWith(canvas)
      resolve(canvas.toDataURL('image/jpeg', %f))
    }
  })
  </script>
  """

  def take_photo(quality=1.0, size=(800,600)):
    display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    f = io.BytesIO(binary)
    img = np.asarray(Image.open(f))

    timestampStr = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
    filename = 'raw_images/photo_%s.jpeg' %timestampStr
    Image.fromarray(img).save(filename)
    print('Image captured and saved to %s' %filename)

  img = take_photo() # click the image to capture a frame!

## list

In [0]:
# List the contents of our image folder:

import os
from PIL import Image

imgs = sorted(os.listdir('raw_images'))

print("Found %d images in %s" %(len(imgs), 'raw_images'))
if len(imgs) == 0:
  print("No image found !!!")
else:
  print(imgs)

  for img_name in imgs:
    img_path = 'raw_images/' + img_name
    print("img_path: %s" %img_path)
    img = Image.open(img_path)
    w,h = img.size
    rescale_ratio = 256 / min(w,h)
    img = img.resize((int(rescale_ratio*w),int(rescale_ratio*h)), Image.LANCZOS)
    display(img)

In [0]:
# auto-align faces

if pictsInEncoder == 1:
  print("asume aligned images are there. do not align")
else :
  print("align images")
  !python align_images.py raw_images/ aligned_images/ --output_size=1048

In [0]:
# display the aligned images
from PIL import Image

def display_folder_content(folder, res = 256):
  if folder[-1] != '/': folder += '/'
  for i, img_path in enumerate(sorted(os.listdir(folder))):
    if '.png' in img_path:
      display(Image.open(folder+img_path).resize((res,res)), 'img %d: %s' %(i, img_path))
      print('\n')
      
display_folder_content('aligned_images')

In [0]:
# show images for encoding 

print("aligned_images contains %d images ready for encoding!" %len(os.listdir('aligned_images/')))

In [0]:
# clean up the videos-to-generate folder

if pictsInEncoder == 1:
  print("asume videos ok. do not remove")
else :
  print("remove pre videos")
  !rm -f videos/* 

# encode

In [0]:
# encode aligned images into latent vectors

# will remove latent vectors and generated images

# will download the StyleGAN network from NVIDIA trained on faces 
# karras2019stylegan-ffhq-1024x1024.pkl
# !gdown https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ

# Set the batch size to the number of images in the aligned_images/ folder
# If the results don't look great: Play with the encoding arguments!!!
# > 1. More iterations             eg.  1000 (def: 200))
# > 2. Decrease the L1 penalty to  eg.   0.1 (def: 0.3))
# > 3. Lower initial learning rate eg. 0.005 (def: 0.01) 
#      Decay_rate (def: 0.8)
# > 4. Find out about the other encoding options here: 
# https://github.com/pbaylies/stylegan-encoder/blob/master/encode_images.py 


if pictsInEncoder == 1:
  print("asume  generated_images and latent_representation ok. do not encode")
else :  
  !rm -rf generated_images  latent_representations
  !python encode_images.py \
  --batch_size=2 \
  --output_video=True \
  --load_resnet='data/finetuned_resnet.h5' \
  --lr=0.005 \
  --decay_rate=0.8 \
  --iterations=400 \
  --use_l1_penalty=0.2 \
  --model_url='https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' \
  aligned_images/ generated_images/ latent_representations/

  print("\n************ Latent code optimization finished! ***************")

In [0]:
# if [content]/gdrive, assuming in [content]/stylegan-encoder


if pictsInEncoder == 1:
  print("asume latent vects ok. do not try to get from gdrive")
else: 
  folder = '../gdrive/My Drive/gdata/'
  isExist = os.path.exists(folder)
  if isExist:
    print ('cp generated_images latent_representations to gdrive')
    !cp -r 'generated_images/' folder
    !cp -r 'latent_representations/' folder

    !ls folder

# model

## model networks (_G, _D, Gs)

In [0]:
# assume in stylegan folder

print("assume in stylegan folder")
!pwd

In [0]:
import dnnlib
import dnnlib.tflib as tflib

print("set model networks")

tflib.init_tf()
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, \
                                              nchw_to_nhwc=True), \
                                              minibatch_size=1)


In [0]:
# load the StyleGAN network - should be in cwd stylegan-encoder
#      assumes karras2019stylegan-ffhq-1024x1024.pkl already in cache

import pickle

model_dir = 'cache/'
model_path = [model_dir+f for f in os.listdir(model_dir) if 'stylegan-ffhq' in f][0]
print("Loading StyleGAN model from %s..." %model_path)

with dnnlib.util.open_url(model_path) as f:
  
  # generator_network, discriminator_network, averaged_generator_network = pickle.load(f)
  _G, _D, Gs = pickle.load(f)
  
print("StyleGAN loaded & ready for sampling!")

# from encoder.generator_model import Generator
# generator = Generator(Gs, batch_size=1, randomize_noise=False) 
  
# print("generator %s" %generator)
   

## image generation functions

In [0]:

# define functions to generate images from latent vectors
#    w (latent representation) or z (random image)

def gen_images(network, latent_vector, z = True):
  
  batch_size = latent_vector.shape[0]   
  #Start from z: run the full network
  if z:
    latent_vector = latent_vector.reshape((batch_size, 512))
    img_array = network.run( \
                       latent_vector, \
                       None, \
                       randomize_noise=False, \
                       **synthesis_kwargs)
  
  #Start from w: skip the mapping network
  else: 
    latent_vector = latent_vector.reshape((batch_size, 18, 512))
    img_array = network.components.synthesis.run( \
                      latent_vector, \
                      randomize_noise=False, \
                      **synthesis_kwargs)
    
  return img_array

   

In [0]:

# generator from Generator(Gs, batch_size=1, randomize_noise=False)

def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    
    generator.set_dlatents(latent_vector)    
    img_array = generator.generate_images()[0]
    
    img = PIL.Image.fromarray(img_array, 'RGB')
    #return img
    return img.resize((256, 256))

def move_and_show(latent_vector, direction, coeffs):
    print("move_and_show")
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
        
        ax[i].imshow(generate_image(new_latent_vector))
        
        ax[i].set_title('Coeff: %0.1f' % coeff)
    [x.axis('off') for x in ax]
    plt.show()
    
    
def generate_image_for_video(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    
    return img_array
  
  
def move_for_video(latent_vector, direction, coeff):
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
    
    img_array = generate_image(new_latent_vector)
    
    return img_array  

# show

In [0]:
# show the encoded images

for f in sorted(os.listdir('latent_representations')):
  
  # gen image from latent vector in latent_representations
  w = np.load('latent_representations/' + f).reshape((1,18,-1))
  
  # use Gs as generator
  img = gen_images(Gs, w, z = False)[0]
  
  plt.imshow(img)
  plt.axis('off')
  plt.title("Generated image from %s" %f)
  plt.show()

In [0]:
#  show the encoded samples side by side with the original ones:

import matplotlib.pyplot as plt

def plot_two_images(img1,img2, img_id, fs = 12):
  f, axarr = plt.subplots(1,2, figsize=(fs,fs))
  axarr[0].imshow(img1)
  axarr[0].title.set_text('Encoded img %d' %img_id)
  axarr[1].imshow(img2)
  axarr[1].title.set_text('Original img %d' %img_id)
  plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
  plt.show()

def display_sbs(folder1, folder2, res = 256):
  if folder1[-1] != '/': folder1 += '/'
  if folder2[-1] != '/': folder2 += '/'
    
  imgs1 = sorted([f for f in os.listdir(folder1) if '.png' in f])
  imgs2 = sorted([f for f in os.listdir(folder2) if '.png' in f])
  if len(imgs1)!=len(imgs2):
    print("Found different amount of images in aligned vs raw image directories. That's not supposed to happen...")
  
  for i in range(len(imgs1)):
    img1 = Image.open(folder1+imgs1[i]).resize((res,res))
    img2 = Image.open(folder2+imgs2[i]).resize((res,res))
    plot_two_images(img1,img2, i)
    print("")
     
display_sbs('generated_images/', 'aligned_images/', res = 512)

In [0]:
# list videos

videos_dir = 'videos'  
videos_dir_bck = 'videos'  

!ls $videos_dir

thereisbck = os.path.isdir(videos_dir_bck)
print("thereisbck %s" %(thereisbck))

if thereisbck == True:
  print("video folders with first names: %s" %(videos_dir_bck))
else:
  print("copy to bck")
  !cp -r $videos_dir $videos_dir_bck

vids = sorted(os.listdir(videos_dir))
print("Found %d videos in %s" %(len(vids), videos_dir))
for i, vid_fullname in enumerate(vids):
  vid_name = vid_fullname.split('.')[0]
  vid_ext = vid_fullname.split('.')[1]
  newvid_fullname = 'vid_%02d.%s' %(i, vid_ext)
  os.rename(videos_dir + '/' + vid_fullname, videos_dir + '/' + newvid_fullname)
!ls $videos_dir

In [0]:
# display videos

from moviepy.editor import *
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.Clip import Clip
from IPython.display import display

if displayInVideo == 1:
  videos_dir = 'videos'  
  vids = sorted(os.listdir(videos_dir))
  for i, vid in enumerate(vids):
    vid_path = videos_dir + '/' + vid
    print("vid %d : %s" %(i, vid_path))
    clip = VideoFileClip(vid_path)
    display(clip.ipython_display(height=512, autoplay=1, loop=1))
else:
  print("will not display in notebook videos")

In [0]:
videos_dir = 'videos'  
from google.colab import files
vids = sorted([f for f in os.listdir(videos_dir)])
print("vids: %s" %vids)
if downloadResults == 1:
  for i, vid_name in enumerate(vids):
    vid_path = videos_dir + "/" + vid_name  
    print("vid_path: %s" %vid_path)  
    files.download(vid_path) 
 

## get good images and select latent vectors

In [0]:
# assume in encoder

print("assume in stylegan-encoder")
!pwd

In [0]:
# select the good latent images

folder = 'latent_representations'

good_images = []
for i, latents in enumerate(sorted(os.listdir(folder))):
  good_images.append(i)

# or set good images from list
# good_images = [0,1,2,3]  # Uncomment and pick out latents that worked well


# save the array of good latent vectors to output_vectors.npy

out_file = '../output_vectors.npy'

import numpy as np
latents = sorted(os.listdir('latent_representations'))

final_w_vectors = []
for img_id in good_images:
  w = np.load('latent_representations/' + latents[img_id])
  final_w_vectors.append(w)

final_w_vectors = np.array(final_w_vectors)
np.save(out_file, final_w_vectors)
print("%d latent vectors of shape %s saved to %s!" %(len(good_images), str(w.shape), out_file))

# close



In [0]:
# up to root folder

%cd ..

# sil

In [0]:
import os

# os.kill(os.getpid(), 9)

from numba import cuda
cuda.select_device(0)
cuda.close()

print("cpu_ram_total: %s" %psutil.virtual_memory().total)
print("cpu_ram_avail: %s" %psutil.virtual_memory().available)

# through latent space

In [0]:
# if in InterFaceGAN folder, move up

import os
signature = 'edit.py'

doesExist = os.path.exists(signature)
if doesExist:
  print("move up from InterFaceGAN folder")
  os.chdir('../')
  
print("assume in root")  
!pwd && ls

In [0]:
# load from root the array of latent space vectors of shape (18, 512)

import numpy as np
final_w_vectors = np.load('output_vectors.npy')

print("%d latent vectors of shape %s loaded from %s!" %(final_w_vectors.shape[0], str(final_w_vectors.shape[1:]), 'output_vectors.npy'))

In [0]:
# clone InterFaceGAN

import os
path = 'InterFaceGAN'

facegangit = "https://github.com/ShenYujun/InterFaceGAN.git"

doesExist = os.path.exists(path)
if not doesExist:
  print("InterFaceGAN does not exist. clone %s" %facegangit)
  !git clone $facegangit
else:
  print("InterFaceGAN already exists. will not clone")

In [0]:
# copy latent vectors to InterFaceGAN folder

!cp output_vectors.npy InterFaceGAN

In [0]:
# move to InterFaceGAN

os.chdir('InterFaceGAN')

In [0]:
# download the pretrained StyleGAN FFHQ network from NVIDIA

import os
path = 'karras2019stylegan-ffhq-1024x1024.pkl'

doesExist = os.path.exists(path)
if not doesExist:
  print("Download the pretrained StyleGAN FFHQ network from NVIDIA")
  !gdown https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ
else:
  print("StyleGAN FFHQ exists. will not download")

In [0]:
# copy the pretrained StyleGAN FFHQ network to the models folder

!cp karras2019stylegan-ffhq-1024x1024.pkl models/pretrain/karras2019stylegan-ffhq-1024x1024.pkl

In [0]:
# convert InterFaceGAN model from to tf to pytorch
print ("convert InterFaceGAN model from to tf to pytorch")

latent_direction = 'age'     #### Pick one of ['age', 'eyeglasses', 'gender', 'pose', 'smile']
morph_strength_start = -2    # Controls how strongly we push the face into a certain latent direction (try 1-5)
morph_strength_end = 0.1     # Controls how strongly we push the face into a certain latent direction (try 1-5)
nr_interpolation_steps = 24  # The amount of intermediate steps/frames to render along the interpolation path

boundary_file = 'stylegan_ffhq_%s_w_boundary.npy' %latent_direction
resultsPath = "results/%s" %latent_direction # eg. results/age
print("remove %s" %resultsPath)
!rm -r $resultsPath

boundary = "boundaries/stylegan_ffhq_%s_w_boundary.npy" %latent_direction

!python edit.py \
      -m stylegan_ffhq \
      -o $resultsPath \
      -b $boundary \
      -n 0

In [0]:
# generate the latent interpolations and output to output_vectors.npy

latent_direction = 'age'     #### Pick one of ['age', 'eyeglasses', 'gender', 'pose', 'smile']
morph_strength_start = 1    # Controls how strongly we push the face into a certain latent direction (try 1-5)
morph_strength_end = -2     # Controls how strongly we push the face into a certain latent direction (try 1-5)
nr_interpolation_steps = 24  # The amount of intermediate steps/frames to render along the interpolation path

boundary_file = 'stylegan_ffhq_%s_w_boundary.npy' %latent_direction
resultsPath = "results/%s" %latent_direction # eg. results/age
!rm -r $resultsPath

boundary = "boundaries/stylegan_ffhq_%s_w_boundary.npy" %latent_direction
startDistance = "%.2f" %morph_strength_start
endDistance = "%.2f" %morph_strength_end
steps = "%d" %nr_interpolation_steps

outputVectors = 'output_vectors.npy'

print("generate InterFaceGAN interpolation into %s" %resultsPath)


print("resultsPath: %s" %resultsPath)
print("latent_direction: %s" %latent_direction)
print("boundary: %s" %boundary)
print("startDistance: %s" %startDistance)
print("endDistance: %s" %endDistance)
print("steps: %s" %steps)

!python edit.py \
      -m stylegan_ffhq \
      -b $boundary \
      -s Wp \
      -i $outputVectors \
      -o $resultsPath \
      --start_distance $startDistance \
      --end_distance $endDistance \
      --steps=$steps

In [0]:
# configure video output

out_path = 'output_videos/'

image_folder = 'results/%s' %latent_direction
print ("images folder %s" %image_folder)
video_fps = 12.

In [0]:
# make the videos per latent direction

from moviepy.editor import *
import cv2

images = [img_path for img_path in sorted(os.listdir(image_folder)) if '.jpg' in img_path]
os.makedirs(out_path, exist_ok=True)

prev_id = None
img_sets = []
for img_path in images:
  img_id = img_path.split('_')[0]
  if img_id == prev_id: #append
    img_sets[-1].append(img_path)
    
  else: #start a new img set
    img_sets.append([])
    img_sets[-1].append(img_path)
  prev_id = img_id

print("Found %d image sets!\n" %len(img_sets))
if image_folder[-1] != '/':
  image_folder += '/'

def make_video(images, vid_name):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(vid_name, fourcc, video_fps, (1024, 1024))
    gen = {}
    for img in images:
      video.write(img)
    video.release()
    print('finished '+ vid_name)  

# make video per latent direction
for i in range(len(img_sets)):
  print("Generating video %d..." %i)
  set_images = []
  vid_name = out_path + 'out_video_%s_%02d.mp4' %(latent_direction,i)
  print("make video: %s" %vid_name)
  for img_path in img_sets[i]:
    set_images.append(cv2.imread(image_folder + img_path))

  set_images.extend(reversed(set_images))
  make_video(set_images, vid_name)

In [0]:
# display the resulting videos

if displayInVideo == 1:
  vids = sorted([f for f in os.listdir('output_videos')])
  print ("vids %s" %vids)
  for i, vid_name in enumerate(vids):
    vid_path = 'output_videos/%s' %vid_name
    print ("vid_path %s" %vid_path)
    clip = VideoFileClip(vid_path)
    display(clip.ipython_display(height=512, autoplay=1, loop=1))
else:
  print("will not display in-notebook videos - check displayInVideo")

In [0]:
# would download the videos

from google.colab import files
vids = sorted([f for f in os.listdir('output_videos')])
if downloadResults == 1:
  for i, vid_name in enumerate(vids):
    vid_path = 'output_videos/%s' %vid_name  
    print("vid_path: %s" %vid_path)  
    files.download(vid_path) 
 

In [0]:
# if InterFaceGAN signature, move up to root

signature='edit.py'
isExist = os.path.exists(signature)
if isExist:
  print("in InterFaceGAN. up to root")
  %cd ..

!pwd

# interpolate

In [0]:
# back to stylegan

folder = 'stylegan-encoder'
isExist = os.path.exists(folder)
if isExist:
  print("cwd to %s" %folder)
  os.chdir(folder)

!pwd

## config interpolation

In [0]:
# Load learned latent directions

import numpy as np
from google.colab import files

smile_direction = np.load('ffhq_dataset/latent_directions/smile.npy')
age_direction = np.load('ffhq_dataset/latent_directions/age.npy')

folder = 'latent_representations/'
vectors = sorted([f for f in os.listdir(folder)])
for i, vector in enumerate(vectors):
  vector_path = folder + vector  
  print("vector: %s" %vector_path)  

    
p0 = folder + vectors[0]
p1 = folder + vectors[1]

v0 = np.load(p0)
v1 = np.load(p1)


In [0]:
# assume in root, back to the encoder

signature='stylegan-encoder'
isExist = os.path.exists(signature)
if isExist:
  print("go to stylegan")
  os.chdir('stylegan-encoder')

print("should be in stylegan-encoder")
!pwd


Traits can be adjusted with the move_and_show() function, which takes three arguments: 
- a latent vector, 
- a latent direction, and 
- an array of coefficients. 

It then performs the following computation:
- new latent vector=latent vector+(coefficient∗latent direction)

The new latent vectors are passed through the StyleGAN genertor, and the resulting images are plotted

## move and show

In [0]:

fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

# bsize = 8
bsize = 1
synthesis_kwargs = dict(output_transform=fmt, minibatch_size=bsize) 


### gen triplet

In [0]:
# move_and_show(v0, smile_direction, [-1, 0, 1])

ere_latent_vector = v0
direction = smile_direction
coeffs = [-1, 0, 1]


# w = v0.copy()
# w = w.reshape((1, 18, 512))
# img = gen_images(Gs, w, z = False)[0]

# model = Gs

print("move_and_show")
fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
for i, coeff in enumerate(coeffs):

  batch_size = 1
  latent_vector = ere_latent_vector.copy()
  latent_vector = latent_vector.reshape((batch_size, 18, -1))
  latent_vector[:8] = (latent_vector + coeff*direction)[:8]

  # use Gs as generator
  img = gen_images(Gs, latent_vector, z = False)[0]
  
  ax[i].imshow(img)
  ax[i].set_title('Coeff: %0.1f' % coeff)

[x.axis('off') for x in ax]
plt.show()


In [0]:
# move_and_show(v1, age_direction, [-4, 0, 2])


ere_latent_vector = v1
direction = age_direction
coeffs = [-4, 0, 2]

# model = Gs

print("move_and_show")
fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
for i, coeff in enumerate(coeffs):

  batch_size = 1
  latent_vector = ere_latent_vector.copy()
  latent_vector = latent_vector.reshape((batch_size, 18, -1))
  latent_vector[:8] = (latent_vector + coeff*direction)[:8]

  # use Gs as generator
  img = gen_images(Gs, latent_vector, z = False)[0]
  
  ax[i].imshow(img)
  ax[i].set_title('Coeff: %0.1f' % coeff)

[x.axis('off') for x in ax]
plt.show()


characters together to find an "average" of the two
 take an average of the two latent vectors and generate an image using the average latent vector.
 adjust how much of each character you want by changing `alpha`.

Now for the moment of truth -- Let's see what the future protector of the realm will look like!

In [0]:
# assume defined iter generator

alpha = 0.5
mix = (((alpha)*v0)+((1-alpha)*v1))
# move_and_show(iter, mix, age_direction, [-2, 0, 2])


ere_latent_vector = mix
direction = age_direction
coeffs = [-2, 0, 2]

# model = Gs

print("move_and_show")
fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
for i, coeff in enumerate(coeffs):

  batch_size = 1
  latent_vector = ere_latent_vector.copy()
  latent_vector = latent_vector.reshape((batch_size, 18, -1))
  latent_vector[:8] = (latent_vector + coeff*direction)[:8]

  # use Gs as generator
  img = gen_images(Gs, latent_vector, z = False)[0]
  
  ax[i].imshow(img)
  ax[i].set_title('Coeff: %0.1f' % coeff)

[x.axis('off') for x in ax]
plt.show()

the new latent character can navigate through age, gender, and smile dimensions

fuse characters through Nvidia's style mixing technique. see article. The code below is a modification of Nvidia's style mixing implementation

In [0]:
def draw_style_mixing_figure(png, Gs, w, h, src_dlatents, dst_dlatents, style_ranges):
    print(png)
    #src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
    #dst_dlatents = Gs.components.mapping.run(dst_latents, None)
    src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
    dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)

    canvas = PIL.Image.new('RGB', (w * (len(src_dlatents) + 1), h * (len(dst_dlatents) + 1)), 'white')
    for col, src_image in enumerate(list(src_images)):
        canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
    for row, dst_image in enumerate(list(dst_images)):
        canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))
        row_dlatents = np.stack([dst_dlatents[row]] * len(src_dlatents))
        row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
        for col, image in enumerate(list(row_images)):
            canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h))
    canvas.save(png)
    return canvas.resize((512,512))

In [0]:
tflib.init_tf()
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=1)
_Gs_cache = dict()

# config.result_dir
!mkdir -p 'outresults'
result_dir = 'outresults'

draw_style_mixing_figure(os.path.join(result_dir, 'style-mixing.png'), \
                         Gs, \
                         w=1024, h=1024, src_dlatents=v0.reshape((1, 18, 512)), \
                         dst_dlatents=v1.reshape((1, 18, 512)), \
                         style_ranges=[range(6,14)])

the simple averaging method seems to produce better results;
try Nvidia's style mixing method with different paramets

Interpolation Videos
GAN interpolation videos perform smooth transitions between various images

### config output video

In [0]:
# config video

duration_sec = 10.0
smoothing_sec = 1.0
mp4_fps = 20
num_frames = int(np.rint(duration_sec * mp4_fps))


# seconds times frames
sf = int(duration_sec * mp4_fps)


### generate videos

In [0]:
# generate videos. saved under the reults folder
# If you're interpolating between two characters, set these characters here

# assume sf defined seconds times frames

# v0
# v1

!mkdir -p 'results'

# This creates an nd array that stores all the image frames for cross-character interpolation
# src_images = np.stack(generate_image_for_video((0.01*alpha*char2)+((1-(0.01*alpha))*char1)) for alpha in range (100))


import numpy as np
import matplotlib.pyplot as plt
def inter(a, x0 = 0, x1 = 2*np.pi, n = sf):
  x = x0 + (x1 - x0 ) * a / n
  y = 0.5 * ( 1 + np.cos(x))
  return y

def a(s,w):
  return inter(s) * w
def b(s,w):
  return (1 - inter(s)) * w

def ab(s, w0, w1):
  return a(s, w0) + b(s, w1)


def generate_image(network, latent_vector, z = True):
    img_array = network.components.synthesis.run( \
                      latent_vector, \
                      randomize_noise=False, \
                      **synthesis_kwargs)
    return img_array

def generate_frame(latent_vector):
    new_latent_vector = latent_vector.copy()
    new_latent_vector = new_latent_vector.reshape((1, 18, -1))
    img = generate_image(Gs, latent_vector, z = False)[0]  
    return img
  
def move_frames(latent_vector, direction, coeff):
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
    
    img_array = generate_frames(new_latent_vector)
    
    return img_array    
  
  

def tmp(w):
  batch_size = 1
  latent_vector = w.copy()
  latent_vector = latent_vector.reshape((batch_size, 18, -1))
  latent_vector[:8] = latent_vector[:8]
  img = gen_images(Gs, latent_vector, z = False)[0]
  return img  

  
# This creates an nd array that stores all the image frames for cross-character interpolation
# src_images = np.stack(generate_image_for_video((0.01*alpha*char2)+((1-(0.01*alpha))*char1)) for alpha in range (100))
# src_images = np.stack(generate_frame(ab(alpha, v0,v1) ) for alpha in range (sf))
src_images = np.stack(tmp(ab(alpha,v0,v1)) for alpha in range (sf))

  
# Uncomment the next line if you want to do a character transforation video, and choose the arguments as per your requirement
#src_images = np.stack(move_for_video(dany_meme, smile_direction, (0.02*alpha)) for alpha in range (-100,100))


def make_frame(t):
    frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
    src_image = src_images[frame_idx]
    return np.array(src_image)

# Generate video.
import moviepy.editor
mp4_file = 'results/interpolate.mp4'
mp4_codec = 'libx264'
mp4_bitrate = '5M'

video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)

### display output video

In [0]:
# display the resulting videos

from moviepy.editor import *
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.Clip import Clip
from IPython.display import display

vids = sorted([f for f in os.listdir('results')])
print ("vids %s" %vids)
for i, vid_name in enumerate(vids):
  vid_path = 'results/%s' %vid_name
  print ("vid_path %s" %vid_path)
  clip = VideoFileClip(vid_path)
  display(clip.ipython_display(height=512, autoplay=1, loop=1))